## This notebook to solidfy journal cleaning process

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%aimport json
from pathlib import Path
%aimport re
from typing import List, Dict
%aimport os
from xml.sax.saxutils import escape
from pathlib import Path
import logging
from math import floor
from datetime import datetime
%aimport re

In [3]:
from data_processing.text_processing import get_text_from_file, set_working_directory, get_working_directory
from data_processing.text_processing import normalize_quotes

In [4]:
from data_processing.text_processing import write_text_to_file

In [5]:
from data_processing.gpt_processing import (
    token_count, get_active_batches,
    generate_messages, create_jsonl_file_for_batch, start_batch_with_retries,
    get_completed_batches, get_batch_response, set_model_settings, delete_old_files, run_immediate_chat_process,
    get_completion_content
)


In [6]:
from data_processing.xml_processing import split_xml_pages, split_xml_on_pagebreaks, save_pages_to_xml

In [7]:
project_dir = Path("/Users/phapman/Desktop/tnh-scholar/")
data_dir = project_dir / "data_processing"
journal_dir = data_dir / "processed_journal_data"
journal_name = "phat-giao-viet-nam-1956-02"
working_dir = journal_dir / journal_name
cleaned_xml_path = working_dir / f"full_cleaned_{journal_name}.xml"
batch_job_dir = working_dir / "processing_batch_files"
batch_file_path = batch_job_dir / f"clean_batch_{journal_name}.jsonl"
clean_batch_jsonl = working_dir / "clean_batch.jsonl"
ocr_file_to_process = journal_dir / journal_name / f"full_OCR_{journal_name}.xml"
logfile = data_dir / "gpt_processing" / "processing_info.log"

In [8]:
# Set up the logger
def setup_logger(log_file_path):
    """
    Configures the logger to write to a log file and the console.
    """
    # Remove existing handlers
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)

    logging.basicConfig(
        level=logging.DEBUG,
        format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",  # Include logger name
        handlers=[
            logging.FileHandler(log_file_path, encoding="utf-8"),
            logging.StreamHandler()  # Optional: to log to the console as well
        ]
    )

    # Suppress DEBUG/INFO logs for specific noisy modules
    modules_to_suppress = ["httpx", "httpcore", "urllib3", "openai"]
    for module in modules_to_suppress:
        logger = logging.getLogger(module)
        logger.setLevel(logging.WARNING)  # Suppress DEBUG and INFO logs

    
    return logging.getLogger(__name__)

In [9]:
logger = setup_logger(logfile)

In [10]:
user_message_string_clean = """{text}"""

In [11]:
def user_wrap_function_clean(text_block):
    return user_message_string_clean.format(text=text_block)

In [12]:
system_message_clean = """You are a meticulous and consistent world expert at cleaning OCR-generated Vietnamese text. 
You are cleaning pages from a 1950's Buddhist Journal. 
Each line of scanned data will be enclosed in <> brackets. Leave <> brackets in place.
Your goal is to minimally modify the text to generate a cleaned version.
Do not remove any content from the main body of the text. 
Do not change the line formatting. 

You can use the semantic meaning of the text to infer corrections—but make no semantic changes. 
You can also add diacritical marks if they are missing or clearly inaccurate. 
Do not change any proper names, except to add missing diacritical marks or to fix orthographic errors if the context is clear.  

This particular text has a title marker in the footer, "Phat Giao Viet Nam," and also a publishing mark diagonally across the text.  
The publishing mark is "TU VIEN HUE QUANG"  and is faint so only parts of it may appear in some locations in the text. 
Text corresponding to these marks (or part thereof) and page numbers can be omitted.

IMPORTANT: If the page is blank return: blank page 
IMPORTANT: Output the corrected text only with no comments (including ``` xml)"""

In [13]:
def get_max_tokens_for_clean(data: str, factor: float=1, buffer: int=100):
    return floor(token_count(data) * factor) + buffer

In [14]:
text = get_text_from_file(ocr_file_to_process)
pages = split_xml_on_pagebreaks(text)

In [15]:
print(pages[3])

HƯỚNG ĐI
CỦA VĂN HÓA
TỪ ĐỜI thượng - cổ, con người đã phải tranh đấu.
Do một nghiệp cảm từ vô thỉ, con người có một sắc
thân sinh hoạt trong một vũ trụ, giữa một xã - hội loài
người, và bên cạnh vô số các loài hữu tình khác.
Có sinh-mạng thì phải bảo tồn sinh mạng, hoặc bằng
một ý thức, hoặc bằng một bản năng.
Nóng, lạnh, gió, mưa, đói, rét, thú dữ, tai họa...
bao nhiều cái mà con người cần phải chiến thắng để sống
còn. Thiên nhiên là kẻ thù, nhưng cũng là người bạn. Yếu
kém thì thiên nhiên dọa nạt, khôn khéo thì thiên nhiên cung
phụng. Con người nhờ hai bàn tay và một khối óc, đã
khai thác thiên nhiên, đã lợi dụng thiên nhiên, đã bắt buộc
thiên nhiên cung cấp cho mình những thứ cần dùng cho
cuộc sống.
PHẬT-GIÁO VIỆT-NAM
3


In [16]:
def wrap_lines(text: str) -> str:
    """
    Encloses each line of the input text with angle brackets.

    Args:
        text (str): The input string containing lines separated by '\n'.

    Returns:
        str: A string where each line is enclosed in angle brackets.
    
    Example:
        >>> enclose_lines("This is a string with   \n   two lines.")
        '<This is a string with  >\n<    two lines.>'
    """
    return '\n'.join(f"<{line}>" for line in text.split('\n'))

In [17]:
def wrap_all_lines(pages):
    return [wrap_lines(page) for page in pages]

In [18]:
def unwrap_lines(text: str) -> str:
    """
    Removes angle brackets (< >) from encapsulated lines and merges them into 
    a newline-separated string.

    Parameters:
        text (str): The input string with encapsulated lines.

    Returns:
        str: A newline-separated string with the encapsulation removed.
    
    Example:
        >>> merge_encapsulated_lines("<Line 1> <Line 2> <Line 3>")
        'Line 1\nLine 2\nLine 3'
        >>> merge_encapsulated_lines("<Line 1>\n<Line 2>\n<Line 3>")
        'Line 1\nLine 2\nLine 3'
    """
    # Find all content between < and > using regex
    matches = re.findall(r"<(.*?)>", text)
    # Join the extracted content with newlines
    return '\n'.join(matches)

In [19]:
def unwrap_all_lines(pages):
    result = []
    for page in pages:
        if page == "blank page":
            result.append(page)
        else:
            result.append(unwrap_lines(page))
    return result

In [20]:
def generate_clean_batch(
    input_xml_file: str,
    output_file: str,
    system_message: str,
    user_wrap_function,
    immediate: bool = False
):
    """
    Generate a batch file for the OpenAI (OA) API using a single input XML file.

    Parameters:
        batch_file (str): Full path to the input XML file to process.
        output_file (str): Full path to the output batch JSONL file.
        system_message (str): System message template for batch processing.
        user_wrap_function (callable): Function to wrap user input for processing pages.

    Returns:
        str: Path to the created batch file.

    Raises:
        Exception: If an error occurs during file processing.
    """

    try:
        # Read the OCR text from the batch file
        text = get_text_from_file(input_xml_file)
        logger.info(f"Processing file: {input_xml_file}")

        # Split the text into pages for processing
        pages = split_xml_on_pagebreaks(text)
        pages =  wrap_all_lines(pages) # wrap lines with brackets.
        if not pages:
            raise ValueError(f"No pages found in XML file: {input_xml_file}")
        logger.info(f"Found {len(pages)} pages in {input_xml_file}.")

        max_tokens = [get_max_tokens_for_clean(page) for page in pages]

        # Generate messages for the pages
        batch_message_seq = generate_messages(system_message, user_wrap_function, pages)

        if immediate:
            logger.info("Running immediate chat process for cleaning:")
            for message in batch_message_seq:
                logger.info("Starting page {i+1}...")
                result = run_immediate_chat_process(batch_message_seq, max_token_list=max_tokens)
                
            
        # Save the batch file
        create_jsonl_file_for_batch(batch_message_seq, output_file, max_token_list=max_tokens)
        logger.info(f"Batch file created successfully: {output_file}")

        return output_file

    except FileNotFoundError:
        logger.error(f"File not found.")
        raise
    except ValueError as e:
        logger.error(f"Value error: {e}")
        raise
    except Exception as e:
        logger.error(f"Unexpected error while processing {input_xml_file}: {e}")
        raise

In [21]:
def run_immediate_clean(
    input_xml_file: str,
    system_message: str,
    user_wrap_function,
):
    try:
        result_list = []
        # Read the OCR text from the batch file
        text = get_text_from_file(input_xml_file)
        logger.info(f"Processing {input_xml_file} for immediate cleaning:")

        # Split the text into pages for processing
        pages = split_xml_on_pagebreaks(text)
        pages =  wrap_all_lines(pages) # wrap lines with brackets.
        if not pages:
            raise ValueError(f"No pages found in XML file: {input_xml_file}")
        logger.info(f"Found {len(pages)} pages in {input_xml_file}.")

        max_tokens = [get_max_tokens_for_clean(page) for page in pages]

        # Generate messages for the pages
        batch_message_seq = generate_messages(system_message, user_wrap_function, pages)

        for i, message in enumerate(batch_message_seq):
            logger.info(f"Starting page {i+1}...")
            completion = run_immediate_chat_process(message, max_tokens=max_tokens[i])
            if completion:
                result_list.append(get_completion_content(completion))
            else:
                logger.error("Chat completion failed.")
                raise RuntimeError("Chat could not complete.")
            
        logger.info(f"Cleaning completed successfully.")

        return result_list

    except FileNotFoundError:
        logger.error(f"File not found.")
        raise
    except ValueError as e:
        logger.error(f"Value error: {e}")
        raise
    except Exception as e:
        logger.error(f"Unexpected error while processing {input_xml_file}: {e}")
        raise

In [22]:
model_settings_clean = {
    "gpt-4o": {
        "max_tokens": 1000,
        "temperature": 0
    }
}
set_model_settings(model_settings_clean)

In [23]:
ocr_file_to_process, ocr_file_to_process.exists()

(PosixPath('/Users/phapman/Desktop/tnh-scholar/data_processing/processed_journal_data/phat-giao-viet-nam-1956-02/full_OCR_phat-giao-viet-nam-1956-02.xml'),
 True)

In [24]:
# clean_result = run_immediate_clean(ocr_file_to_process, system_message_clean, user_wrap_function_clean)
# cleaned_data = clean_result

In [26]:
generate_clean_batch(ocr_file_to_process, batch_file_path, system_message_clean, user_wrap_function_clean)
job_description = f"cleaning test for {journal_name} on file: {ocr_file_to_process}"
cleaned_data = start_batch_with_retries(batch_file_path, job_description)

2024-12-02 13:21:29,592 - __main__ - INFO - Processing file: /Users/phapman/Desktop/tnh-scholar/data_processing/processed_journal_data/phat-giao-viet-nam-1956-02/full_OCR_phat-giao-viet-nam-1956-02.xml
2024-12-02 13:21:29,594 - __main__ - INFO - Found 50 pages in /Users/phapman/Desktop/tnh-scholar/data_processing/processed_journal_data/phat-giao-viet-nam-1956-02/full_OCR_phat-giao-viet-nam-1956-02.xml.
2024-12-02 13:21:29,615 - SystemMessageLogger - INFO - System Message:
You are a meticulous and consistent world expert at cleaning OCR-generated Vietnamese text. 
You are cleaning pages from a 1950's Buddhist Journal. 
Each line of scanned data will be enclosed in <> brackets. Leave <> brackets in place.
Your goal is to minimally modify the text to generate a cleaned version.
Do not remove any content from the main body of the text. 
Do not change the line formatting. 

You can use the semantic meaning of the text to infer corrections—but make no semantic changes. 
You can also add diac

In [37]:
# cleanup files:
delete_old_files(datetime.now())

Deleted file: file-VRLZGYCUnmBtJzWQi68Qrh (created on 2024-12-02 07:33:34)
Deleted file: file-AiDSA9nQYZQ8EBSbvvfub8 (created on 2024-12-02 07:25:40)
Deleted file: file-KkN6DohGBiDPKY6ALSQ8QP (created on 2024-12-02 07:22:06)


In [27]:
cleaned_data[5]

'<Trong một trường đấu tranh khốc> <liệt và bao la, dùng sát hại để> <một ảo tưởng> <được.> <ngăn sát hại, là> <không thể thực hiện> <Luật nhân quả không sai chạy> <bao giờ: gieo gió gặt bão, trồng> <ngô được ngô, trồng đậu được đậu,> <và gầy nhân sát hại thì chịu quả> <sát hại.> <Con người phải ý thức được> <điểm hệ trọng đó và phải nhận rằng> <văn hóa đã đi lạc đường.> <Con người cần thấy rằng dục> <vọng tham sân đã gây loạn cho thiên> <hạ, phải quay về tự thân để mở một> <cuộc thanh trừng vĩ đại bên trong nội> <giới. Phải ý thức rằng bản thân mình> <chứa đựng những yếu tố trí tuệ, tình> <thương, mà cũng chứa đựng cả những> <yếu tố si mê dục vọng.> <Dục vọng si mê đã che lấp tình> <thương và trí tuệ: con người phải> <tranh đấu để diệt trừ chúng, để nuôi> <dưỡng trí tuệ và tình thương. Có trí> <tuệ tình thương thì con người mới> <có được hạnh phúc, con người mới> <có thể đoàn kết sâu rộng để tạo một> <đời sống chung cùng tươi đẹp.> <Văn hóa xưa kia đã từ hướng tranh-> <đấu với thiên n

In [28]:
cleaned_data = unwrap_all_lines(cleaned_data)

In [29]:
cleaned_data

['PHẬT-GIÁO\nVIỆT-NAM\nNGUYỆT-SAN\nSỐ 2 RA NGÀY 15 THÁNG 9 BÍNH-THÂN\nTHƯỜNG HỘI PHẬT-GIÁO VIỆT-NAM XUẤT-BẢN',
 'PHẬT-GIÁO\nVIỆT - NAM\nMỤC LỤC Số 2\nRẰM THÁNG 9 BÍNH THÂN\nHƯỚNG ĐI CỦA VĂN HÓA\nXÁC NHẬN GIÁ TRỊ CỦA CON NGƯỜI\nYẾU TỐ NÀO ĐÃ QUYẾT ĐỊNH SỰ\nTHÀNH CÔNG CỦA NGUYỄN-CÔNG TRỨ\nVĂN HỌC PHẬT GIÁO\nMÙA XUÂN ĐẠO\nCUỘC VIẾNG THĂM CÁC PHẬT ĐỊA\nPHẬT - GIÁO VÀ KHOA HỌC\nNHỊP CẦU THÔNG CẢM\nTƯ TƯỞNG HỆ PHẬT GIÁO\nTHOÁT NGỤC VÀNG\nCA-DAO\nP. G. V. N\nDÃ THẢO\nMINH HẠNH\nThầy THẠC-ĐỨC\nTHANH-TI\nThầy THIỆN HÒA\nVIÊN-ĐÌNH\nTHIỀU CHI HOA\nThầy TRÍ-QUANG\nVÕ ĐÌNH CƯỜNG\nTÂM KIẾN',
 'blank page',
 'HƯỚNG ĐI\nCỦA VĂN HÓA\nTỪ ĐỜI thượng - cổ, con người đã phải tranh đấu.\nDo một nghiệp cảm từ vô thỉ, con người có một sắc\nthân sinh hoạt trong một vũ trụ, giữa một xã - hội loài\nngười, và bên cạnh vô số các loài hữu tình khác.\nCó sinh-mạng thì phải bảo tồn sinh mạng, hoặc bằng\nmột ý thức, hoặc bằng một bản năng.\nNóng, lạnh, gió, mưa, đói, rét, thú dữ, tai họa...\nbao nhiêu cái mà con người

In [30]:
save_pages_to_xml(cleaned_xml_path, cleaned_data, overwrite=True)

XML file successfully saved at /Users/phapman/Desktop/tnh-scholar/data_processing/processed_journal_data/phat-giao-viet-nam-1956-02/full_cleaned_phat-giao-viet-nam-1956-02.xml


In [69]:
full_cleaned_text = join_pages(cleaned_data)

In [70]:
print(full_cleaned_text)

<document>
<page page="1">
    PHẬT GIÁO VIỆT NAM NGUYỆT SAN SỐ 1 RA NGÀY 15 THÁNG 8 BÍNH THÂN CH TỔNG HỘI PHẬT GIÁO VIỆT NAM XUẤT.
</page>
<page page="2">
    PHẬT-GIÁO
VIỆT - NAM
TỪ NGÀY Tỳ Ni-Đà-Lưu-Chi sang nước ta đến nay,
kể ra đã đến mười lăm thế kỷ. Phật-Giáo đã ở lại cùng chúng ta một
ngàn năm trăm năm, và đã cùng dân tộc Việt Nam chịu chung bao
nhiêu thăng trầm vinh nhục.
Phật-Giáo Việt Nam quả là một nền Phật-Giáo dân tộc.
PHẬT-GIÁO VIỆT NAM
Phật-Giáo Việt Nam không
phải chỉ là một tôn giáo tín-
ngưỡng mà bất cứ thời nào, ở
đâu, cũng chỉ biết có sứ mệnh
của tôn-giáo tín ngưỡng. Không
ở bất cứ nước nào trên thế giới
cũng vậy, khi bước chân đến,
Đạo Phật cũng thích nghi ngay
với phong tục, khí hậu, nhân
tính để biến thành một lối
sống cho quần chúng. Ở Việt
Nam cũng thế. Phật Giáo
đã hòa hợp trong cá tính dân tộc
ta, đã cùng dân tộc ta xây dựng
một văn hóa quốc gia độc lập.
  </page>
<page page="3">
    Dở lại những trang sử vẻ vang của dân tộc, ta thấy người
Việt luôn luôn có

In [23]:
full_cleaned_path = journal_dir / basename / f"full_cleaned_{basename}.xml"
full_cleaned_path

PosixPath('/Users/phapman/Desktop/tnh-scholar/data_processing/processed_journal_data/phat-giao-viet-nam-1956-01/full_cleaned_phat-giao-viet-nam-1956-01.xml')

In [ ]:
#write_text_to_file(full_cleaned_path, full_cleaned_text)

In [24]:
full_cleaned_current = get_text_from_file(full_cleaned_path)

In [25]:
print(full_cleaned_current)

<document>
    <page page="1">
        PHẬT GIÁO VIỆT NAM NGUYỆT SAN SỐ 1 RA NGÀY 15 THÁNG 8 BÍNH THÂN CH TỔNG HỘI PHẬT GIÁO VIỆT
        NAM XUẤT.
    </page>
    <page page="2">
        PHẬT-GIÁO
        VIỆT - NAM
        TỪ NGÀY Tỳ Ni-Đà-Lưu-Chi sang nước ta đến nay,
        kể ra đã đến mười lăm thế kỷ. Phật-Giáo đã ở lại cùng chúng ta một
        ngàn năm trăm năm, và đã cùng dân tộc Việt Nam chịu chung bao
        nhiêu thăng trầm vinh nhục.
        Phật-Giáo Việt Nam quả là một nền Phật-Giáo dân tộc.
        PHẬT-GIÁO VIỆT NAM
        Phật-Giáo Việt Nam không
        phải chỉ là một tôn giáo tín-
        ngưỡng mà bất cứ thời nào, ở
        đâu, cũng chỉ biết có sứ mệnh
        của tôn-giáo tín ngưỡng. Không
        ở bất cứ nước nào trên thế giới
        cũng vậy, khi bước chân đến,
        Đạo Phật cũng thích nghi ngay
        với phong tục, khí hậu, nhân
        tính để biến thành một lối
        sống cho quần chúng. Ở Việt
        Nam cũng thế. Phật Giáo
        đã hòa hợp 

In [107]:
token_count(full_cleaned_current)

35222

In [22]:
cleaned_pages = split_xml_pages(full_cleaned_current)

In [23]:
cleaned_pages

['<page page="1">\n        PHẬT GIÁO VIỆT NAM NGUYỆT SAN SỐ 1 RA NGÀY 15 THÁNG 8 BÍNH THÂN CH TỔNG HỘI PHẬT GIÁO VIỆT\n        NAM XUẤT.\n    </page>\n    ',
 '<page page="2">\n        PHẬT-GIÁO\n        VIỆT - NAM\n        TỪ NGÀY Tỳ Ni-Đà-Lưu-Chi sang nước ta đến nay,\n        kể ra đã đến mười lăm thế kỷ. Phật-Giáo đã ở lại cùng chúng ta một\n        ngàn năm trăm năm, và đã cùng dân tộc Việt Nam chịu chung bao\n        nhiêu thăng trầm vinh nhục.\n        Phật-Giáo Việt Nam quả là một nền Phật-Giáo dân tộc.\n        PHẬT-GIÁO VIỆT NAM\n        Phật-Giáo Việt Nam không\n        phải chỉ là một tôn giáo tín-\n        ngưỡng mà bất cứ thời nào, ở\n        đâu, cũng chỉ biết có sứ mệnh\n        của tôn-giáo tín ngưỡng. Không\n        ở bất cứ nước nào trên thế giới\n        cũng vậy, khi bước chân đến,\n        Đạo Phật cũng thích nghi ngay\n        với phong tục, khí hậu, nhân\n        tính để biến thành một lối\n        sống cho quần chúng. Ở Việt\n        Nam cũng thế. Phật Giáo\n  

In [24]:
import re

def remove_page_tags(text):
    """
    Removes <page ...> and </page> tags from a text string.

    Parameters:
    - text (str): The input text containing <page> tags.

    Returns:
    - str: The text with <page> tags removed.
    """
    # Remove opening <page ...> tags
    text = re.sub(r"<page[^>]*>", "", text)
    # Remove closing </page> tags
    text = re.sub(r"</page>", "", text)
    return text

In [25]:
cleaned_pages = [remove_page_tags(page) for page in cleaned_pages]

In [26]:
cleaned_pages[1]

'\n        PHẬT-GIÁO\n        VIỆT - NAM\n        TỪ NGÀY Tỳ Ni-Đà-Lưu-Chi sang nước ta đến nay,\n        kể ra đã đến mười lăm thế kỷ. Phật-Giáo đã ở lại cùng chúng ta một\n        ngàn năm trăm năm, và đã cùng dân tộc Việt Nam chịu chung bao\n        nhiêu thăng trầm vinh nhục.\n        Phật-Giáo Việt Nam quả là một nền Phật-Giáo dân tộc.\n        PHẬT-GIÁO VIỆT NAM\n        Phật-Giáo Việt Nam không\n        phải chỉ là một tôn giáo tín-\n        ngưỡng mà bất cứ thời nào, ở\n        đâu, cũng chỉ biết có sứ mệnh\n        của tôn-giáo tín ngưỡng. Không\n        ở bất cứ nước nào trên thế giới\n        cũng vậy, khi bước chân đến,\n        Đạo Phật cũng thích nghi ngay\n        với phong tục, khí hậu, nhân\n        tính để biến thành một lối\n        sống cho quần chúng. Ở Việt\n        Nam cũng thế. Phật Giáo\n        đã hòa hợp trong cá tính dân tộc\n        ta, đã cùng dân tộc ta xây dựng\n        một văn hóa quốc gia độc lập.\n    \n    '

In [38]:
cleaned_pages[8]

'\n        đề tự tiến bộ, để tự giải phóng thì\n        quả thật giá trị con người không là\n        bao lăm cả. Ý chí và năng lực\n        chúng ta sẽ tiêu mòn, nếu ta nhận\n        thấy ta bất lực, nếu ta nhận thấy\n        ta không có quyền gì vượt khỏi ý\n        muốn của một đảng tối cao và linh\n        thiêng.\n        Con người do nghiệp lực quá\n        khứ\n        và hiện tại mà có một sắc thân\n        sinh hoạt trong một hoàn cảnh xã\n        hội. Đạo Phật dạy rằng con người\n        phải cải thiện nghiệp nhân để có\n        một nghiệp quả tốt đẹp hơn.\n        Nghiệp quả tốt đẹp ấy chính là một\n        con người tốt sống trong một hoàn\n        cảnh đẹp. Sự tốt đẹp này không phải do\n        một đảng thiêng liêng nào ban xuống mà\n        do ở chính sự chuyển nghiệp của\n        con người.\n        Có những kẻ không mê tín các\n        lực lượng siêu nhiên nhưng lại mê\n        tín ở năng lực rèn đúc của một tổ\n        chức xã hội. Họ bảo: xã hội tốt\n        đẹp sẽ đào